In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.12:3.4.0 --conf spark.cassandra.connection.host=172.20.0.2 pyspark-shell'

In [2]:
import json
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, FloatType, ArrayType
from pyspark.sql.functions import col, explode
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.master('local[12]').appName("btc_insertion").getOrCreate()

23/08/06 06:59:32 WARN Utils: Your hostname, Obi-Wan resolves to a loopback address: 127.0.1.1; using 192.168.1.2 instead (on interface enp6s0)
23/08/06 06:59:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/narvy/.ivy2/cache
The jars for the packages stored in: /home/narvy/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa99b634-9c1b-4f1d-a534-863496b30775;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central


:: loading settings :: url = jar:file:/home/narvy/Avatarify/miniconda3/envs/data-infra/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.github.spotbugs#spotbugs-annotations;3.1.12 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.datastax.oss#java-driver-mapper-runtime;4.13.0 in central
	found com.datastax.oss#java-driver-query-builder;4.13.0 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.scala-lang#scala-reflect;2.12.11 in central
:: resolution report :: resolve 275m

In [3]:
input_schema = StructType([
        StructField("ot", ArrayType(FloatType())),
        StructField("ct", ArrayType(FloatType())),
        StructField("o", ArrayType(FloatType())),
        StructField("h", ArrayType(FloatType())),
        StructField("l", ArrayType(FloatType())),
        StructField("c", ArrayType(FloatType())),
        StructField("v", ArrayType(FloatType()))
])

data = json.load(open('../data/dataset.json', 'r'))

In [ ]:
d_frames = {}

conf = SparkConf().setAppName("df_cassandra")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

for interval in data:
    data_i = data[interval]
    df = spark.createDataFrame([Row(
        ot = data_i['ot'],
        ct = data_i['ct'],
        o = data_i['o'],
        h = data_i['h'],
        l = data_i['l'],
        c = data_i['c'],
        v = data_i['v']
    )], schema=input_schema)
    
    df = df\
        .withColumn('ot', explode(df['ot']))\
        .withColumn('ct', explode(df['ct']))\
        .withColumn('o', explode(df['o']))\
        .withColumn('h', explode(df['h']))\
        .withColumn('l', explode(df['l']))\
        .withColumn('c', explode(df['c']))\
        .withColumn('v', explode(df['v']))
    
    d_frames[interval] = df = df.withColumn('ot', col('ot').cast("Timestamp")).withColumn('ct', col('ct').cast("Timestamp"))
    
    partitions = 100
    p_len = df.count() // n_splits
    
    for s in range(partitions):
        parted_df = df.limit(p_len)
        df = df.subtract(parted_df)
    
        parted_df.write\
            .format("org.apache.spark.sql.cassandra")\
            .options(table = f'raw_{interval}', keyspace = 'bitcoin_chart')\
            .mode('append')\
            .save()
    

23/08/06 06:59:44 WARN TaskSetManager: Stage 0 contains a task of very large size (188491 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:

# Define the number of splits you want
n_splits = 4
  
# Calculate count of each dataframe rows
each_len = prod_df.count() // n_splits
  
# Create a copy of original dataframe
copy_df = prod_df
  
# Iterate for each dataframe
i = 0
while i < n_splits:
  
    # Get the top `each_len` number of rows
    temp_df = copy_df.limit(each_len)
  
    # Truncate the `copy_df` to remove
    # the contents fetched for `temp_df`
    copy_df = copy_df.subtract(temp_df)
  
    # View the dataframe
    temp_df.show(truncate=False)
  
    # Increment the split number
    i += 1